1. 加载数据
2. 搜索最佳参数
3. 可视化
4. 存标签

In [1]:
import os
import shutil
from tqdm import tqdm
import random
import math
from glob import glob

import pickle as pkl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import ParameterGrid
from sklearn.cluster import KMeans


In [2]:
os.getcwd()

'/data1/home/xiruiling/course/AdvanceArtificialIntelligence/AAI_Proj/script'

In [3]:
# get the current directory
# proj_root_dir = '/data1/home/xiruiling/course/AdvanceArtificialIntelligence/AAI_Proj'
proj_root_dir = os.path.abspath(os.path.join(os.getcwd(),".."))
print("proj_root_dir: ", proj_root_dir)

proj_root_dir:  /data1/home/xiruiling/course/AdvanceArtificialIntelligence/AAI_Proj


In [15]:
# load
tSNE_result_dir = os.path.join(proj_root_dir, 'state_dict', 'tSNE_3D.pkl')
with open(tSNE_result_dir, 'rb') as f:
    tSNE_result = pkl.load(f)

idx_dir = os.path.join(proj_root_dir, 'state_dict', 'fft_result_wo_simplify', 'trnIdx.pkl')
with open(idx_dir, 'rb') as f:
    Idx = pkl.load(f)

# print(tSNE_result)
print(len(Idx))

pd.reset_option('display.float_format')
tSNE_result_pd = pd.DataFrame(tSNE_result)
tSNE_result_pd


251


,0,1,2
0,2.697207e-06,2.175114e-06,-1.444606e-06
1,-1.820185e-06,1.996362e-06,-1.543234e-07
2,8.055199e-07,3.526720e-06,1.933964e-06
3,-1.747895e-06,2.983067e-06,-7.219960e-07
4,-3.069659e-06,4.365137e-06,1.587823e-06
...,...,...,...
890827,-4.436323e-07,-4.743703e-06,2.518872e-06
890828,5.872241e-06,-8.220618e-07,-6.376421e-06
890829,4.336947e-06,-1.994195e-06,-6.970361e-06
890830,4.427898e-06,-8.360230e-06,-6.005343e-06


In [89]:
# Search parameters
parameters = [16, 12, 20, 24]

# instantiating ParameterGrid, pass number of clusters as input
parameter_grid = ParameterGrid({'n_clusters': parameters})

best_score = -1
kmeans_model = KMeans()     # instantiating KMeans model
silhouette_scores = []

# evaluation based on silhouette_score
for p in tqdm(parameter_grid):
    kmeans_model.set_params(**p)    # set current hyper parameter
    kmeans_model.fit(tSNE_result_pd)          # fit model on wine dataset, this will find clusters based on parameter p
    with open(os.path.join('/data1/home/xiruiling/course/AdvanceArtificialIntelligence/AAI_Proj/state_dict/knn_model', 'knn_'+str(p['n_clusters'])+'_.pkl'), 'wb') as f:
        pkl.dump(kmeans_model, f)
    ss = metrics.silhouette_score(tSNE_result_pd, kmeans_model.labels_)   # calculate silhouette_score
    silhouette_scores += [ss]       # store all the scores

    print('Parameter:', p, 'Score', ss)

    # check p which has the best score
    if ss > best_score:
        best_score = ss
        best_grid = p

# plotting silhouette score
plt.bar(range(len(silhouette_scores)), list(silhouette_scores), align='center', color='#722f59', width=0.5)
plt.xticks(range(len(silhouette_scores)), list(parameters))
plt.title('Silhouette Score', fontweight='bold')
plt.xlabel('Number of Clusters')
plt.show()

best_grid['n_clusters']

  0%|          | 0/4 [00:00<?, ?it/s]/data1/home/xiruiling/anaconda3/envs/TestEnv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
  0%|          | 0/4 [00:18<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: "/data1/home/xiruiling/course/AdvanceArtificialIntelligence/AAI_Proj/state_dict/knn_model/knn_{'n_clusters': 16}_.pkl"

In [87]:
optimum_num_clusters = 16 #best_grid['n_clusters']
print("optimum num of clusters =", optimum_num_clusters)

optimum num of clusters = 16


In [16]:
# get the centroids
kmeans = KMeans(n_clusters=16)
with open(os.path.join(proj_root_dir, 'state_dict', 'knn_model', 'knn_20_.pkl'), 'rb') as f:
    kmeans = pkl.load(f)
# kmeans.fit(tSNE_result_pd)
centroids = kmeans.cluster_centers_
centroids
# centroids_tSNE = tSNE_result_pd.transform(centroids)

array([[ 2.3504751e-06, -1.7730632e-07, -2.2947165e-06],
       [-5.5951182e-06,  6.4746882e-07,  3.9640358e-07],
       [ 2.1120290e-05, -9.4594925e-06, -1.6061931e-05],
       [ 2.5874872e-06, -7.2141274e-06, -2.5601073e-06],
       [-7.1858426e-07,  1.8908331e-06,  2.5997454e-07],
       [ 3.4716334e-06,  3.5247664e-05, -3.0129366e-05],
       [-4.0009122e-06, -7.3689598e-06,  3.3467427e-06],
       [ 1.7055230e-04,  1.0153509e-04,  1.5247456e-05],
       [-6.2623752e-05,  1.1109019e-05, -2.4256386e-05],
       [ 4.4431545e-05,  1.5437539e-05, -7.0885067e-06],
       [ 2.6224777e-06,  1.2320022e-06,  2.0281198e-06],
       [ 3.8886337e-05, -5.3712574e-05, -5.1309962e-07],
       [-1.9160400e-06, -2.3396021e-06, -2.8191548e-06],
       [-2.7606554e-06,  5.0584026e-06, -4.3753162e-06],
       [ 4.4663334e-06, -3.9149172e-06,  5.7292659e-06],
       [-2.7258479e-06,  6.2248255e-06,  9.2966661e-07],
       [ 6.8330124e-08, -2.2280840e-06,  1.2391743e-06],
       [ 9.3802350e-07, -3.2106

In [5]:
# Visualization

x = tSNE_result[:, 0]
y = tSNE_result[:, 1]

plt.figure(figsize=(10,10))
plt.scatter(x, y, c=kmeans.labels_, alpha=0.5, s=2)  # plot different colors per cluster
plt.scatter(centroids[:,0] , centroids[:,1] , s = 10, marker='X', color = 'red')

plt.title('KNN clusters')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.legend()
# plt.scatter(knn_result[:, 0], knn_result[:, 1], marker='X', s=200, linewidths=1.5,
#             color='red', edgecolors="black", lw=1.5)

plt.show()

NameError: name 'tSNE_result' is not defined

In [17]:
kmeans.labels_
np.unique(np.array(kmeans.labels_))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int32)

In [19]:
labels = kmeans.labels_

final_result = np.zeros((len(Idx)-1, 2))
for i in range(len(Idx)-1):
    final_result[i][0] = i
    majority_vote_record = np.zeros((20))
    for j in range(Idx[i], Idx[i+1]):
        majority_vote_record[labels[j]] = majority_vote_record[labels[j]] + 1
    final_result[i][1] = np.argmax(np.array(majority_vote_record))
    # print(j, "th speaker record: ", majority_vote_record)
    # print("vote: ", np.argmax(np.array(majority_vote_record)), "\n")

np.save(os.path.join(proj_root_dir, 'state_dict', 'knn_specialty'), final_result)
specialty_record = np.zeros((20))
for i in range(len(Idx)-1):
    specialty_record[int(final_result[i][1])] = specialty_record[int(final_result[i][1])] + 1

specialty_record

IndexError: index 16 is out of bounds for axis 0 with size 16